<a href="https://colab.research.google.com/github/GenAIUnplugged/langchain_series/blob/main/05_runnable_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_openai import ChatOpenAI

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o-mini",temperature=0)

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human", "List the main features of the product {product_name}."),
    ]
)


In [ ]:
# Define pros analysis step
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the pros of these features.",
            ),
        ]
    )
    return pros_template.format_prompt(features=features)

In [ ]:
# Define cons analysis step
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the cons of these features.",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)

In [ ]:
# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"

In [ ]:
# Simplify branches with LCEL
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

In [ ]:
cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

In [ ]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

In [ ]:
# Run the chain
result = chain.invoke({"product_name": "MacBook Pro"})
print(result)

Pros:
Certainly! Here are the pros of the features associated with the MacBook Pro:

1. **Display**:
   - **Retina Display with True Tone**: Offers stunning visuals with vibrant colors and sharp details, making it ideal for graphic design, video editing, and media consumption.
   - **High Resolution**: The high pixel density ensures that text and images are incredibly sharp, enhancing the overall viewing experience.
   - **P3 Wide Color Gamut**: Provides a broader range of colors, which is essential for professionals in photography and video production.
   - **High Brightness**: The ability to reach up to 1600 nits peak brightness allows for excellent visibility in various lighting conditions, including bright environments.

2. **Performance**:
   - **Apple Silicon**: The M1, M1 Pro, M1 Max, M2, and M2 Pro chips deliver exceptional performance and efficiency, allowing for smooth multitasking and handling of demanding applications.
   - **High-Performance CPU and GPU**: Ideal for profes